In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import torch
import random

import pandas as pd

from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn

from torch.optim import AdamW

import torch_geometric.transforms as T

from torch_geometric.data import Batch

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_add_pool
from torch_geometric.nn import GraphConv
from torch.utils.data import DataLoader

from pathlib import Path

from tqdm import tqdm

In [11]:
import sys
import os
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
parent_parent_dir = os.path.dirname(parent_dir)

sys.path.append(parent_dir)
sys.path.append(parent_parent_dir)

from DataPipeline.dataset import ZincSubgraphDatasetStep, custom_collate_passive_add_feature
from Model.GNN1 import ModelWithEdgeFeatures
from Model.metrics import pseudo_accuracy_metric, pseudo_recall_for_each_class, pseudo_precision_for_each_class

In [12]:
datapath = Path('..') / '../DataPipeline/data/preprocessed_graph_no_I_Br_P.pt'
dataset = ZincSubgraphDatasetStep(data_path = datapath, GNN_type=1)

Dataset encoded with size 7


In [13]:
loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=custom_collate_passive_add_feature)

In [14]:
encoding_size = 7

model = ModelWithEdgeFeatures(in_channels=encoding_size + 1, hidden_channels_list=[64, 128, 256, 512, 512], mlp_hidden_channels=512, edge_channels=4, num_classes=encoding_size, use_dropout=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=0.0001)

# Set up the loss function for multiclass 
criterion = nn.CrossEntropyLoss()


name = 'GNN1'

In [17]:

# Training function

from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.metrics import mean_squared_error
import numpy as np


def train(loader, epoch):
    model.train()
    total_loss = 0
    mse_sum = 0
    num_correct = 0
    num_correct_recall = torch.zeros(encoding_size)
    num_correct_precision = torch.zeros(encoding_size)
    count_per_class_recall = torch.zeros(encoding_size)
    count_per_class_precision = torch.zeros(encoding_size)
    progress_bar = tqdm_notebook(loader, desc="Training", unit="batch")

    avg_output_vector = np.zeros(encoding_size)  # Initialize the average output vector
    avg_label_vector = np.zeros(encoding_size)  # Initialize the average label vector
    total_graphs_processed = 0

    

    for batch_idx, batch in enumerate(progress_bar):
        data = batch[0]
        terminal_node_infos = batch[1]
        data = data.to(device)
        optimizer.zero_grad()
        logit_out = model(data)
        terminal_node_infos = terminal_node_infos.to(device)

        out = F.softmax(logit_out, dim=1)
        loss = criterion(logit_out, terminal_node_infos)
        num_correct += pseudo_accuracy_metric(out.detach().cpu(), terminal_node_infos.detach().cpu(), random=True)

        recall_output = pseudo_recall_for_each_class(out.detach().cpu(), terminal_node_infos.detach().cpu(), random=True)
        precision_output = pseudo_precision_for_each_class(out.detach().cpu(), terminal_node_infos.detach().cpu(), random=True)
        num_correct_recall += recall_output[0]
        num_correct_precision += precision_output[0]
        count_per_class_recall += recall_output[1]
        count_per_class_precision += precision_output[1]
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
        loss_value = total_loss / (data.num_graphs * (progress_bar.last_print_n + 1))

        # Compute MSE
        mse = mean_squared_error(terminal_node_infos.detach().cpu(), out.detach().cpu())
        mse_sum += mse * data.num_graphs
        mse_value = mse_sum / (data.num_graphs * (progress_bar.last_print_n + 1))

        # Update the average output vector
        avg_output_vector += out.detach().cpu().numpy().mean(axis=0) * data.num_graphs
        avg_label_vector += terminal_node_infos.detach().cpu().numpy().mean(axis=0) * data.num_graphs
        total_graphs_processed += data.num_graphs
        current_avg_output_vector = avg_output_vector / total_graphs_processed
        current_avg_label_vector = avg_label_vector / total_graphs_processed
        avg_correct = num_correct / total_graphs_processed
        avg_correct_recall = num_correct_recall / count_per_class_recall
        avg_correct_precision = num_correct_precision / count_per_class_precision
        avg_f1 = 2 * (avg_correct_recall * avg_correct_precision) / (avg_correct_recall + avg_correct_precision)
        progress_bar.set_postfix(loss=loss_value, mse=mse_value, avg_output_vector=current_avg_output_vector, 
                                 avg_label_vector=current_avg_label_vector, 
                                 avg_correct=avg_correct, num_correct=num_correct, 
                                 total_graphs_processed=total_graphs_processed, 
                                 avg_correct_precision=avg_correct_precision, 
                                 avg_correct_recall=avg_correct_recall, 
                                 avg_f1=avg_f1,
                                 count_per_class_precision=count_per_class_precision,
                                 count_per_class_recall=count_per_class_recall)


    return total_loss / len(loader.dataset), current_avg_label_vector, current_avg_output_vector, avg_correct , avg_correct_precision, avg_correct_recall

In [20]:
# Create a dataframe to save the training history
training_history = pd.DataFrame(columns=['epoch', 'loss', 'avg_output_vector', 'avg_label_vector', 'avg_correct', 'precision', 'recall'])

n_epochs = 250
for epoch in range(1, n_epochs+1):
    loss, avg_label_vector, avg_output_vector, avg_correct, avg_correct_precision, avg_correct_recall = train(loader, epoch)
    training_history.loc[epoch] = [epoch, loss, avg_output_vector, avg_label_vector, avg_correct, avg_correct_precision, avg_correct_recall]

    #save the model(all with optimizer step, the loss ) every 5 epochs

    save_every_n_epochs = 5
    if (epoch) % save_every_n_epochs == 1:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            # Add any other relevant information you want to save here
        }
        torch.save(checkpoint, './history_training/'+f'checkpoint_epoch_{epoch+1}_{name}.pt')
        
    #save the training history every 10 epochs
    if epoch % 1 == 0:
        training_history.to_csv(f"training_history_{name}.csv", index=False)
    print(f'Epoch: {epoch}, Loss: {loss:.8f}')

[autoreload of Model.metrics failed: Traceback (most recent call last):
  File "c:\Users\Anto\anaconda3\envs\torch-geometric\Lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\Anto\anaconda3\envs\torch-geometric\Lib\site-packages\IPython\extensions\autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\Anto\anaconda3\envs\torch-geometric\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 936, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1074, in get_code
  File "<frozen importlib._bootstrap_external>", line 1004, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\Anto\Documents\stage\MolGen\Model\metrics.py", line 26
 

Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

Epoch: 1, Loss: 0.56886243


Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

Epoch: 2, Loss: 0.56595221


...

Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

Epoch: 73, Loss: 0.51602379


Training:   0%|          | 0/1845 [00:00<?, ?batch/s]

Epoch: 74, Loss: 0.51640428


Training:   0%|          | 0/1845 [00:00<?, ?batch/s]